### Claude3 Sonnet LLM

In [52]:
from crewai import Agent, Crew
from crewai import Task
from langchain.chat_models import BedrockChat
import os
import boto3

BEDROCK_CLIENT = boto3.client("bedrock-runtime", 'us-east-1')

llm = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs={"temperature": 0.1}, client=BEDROCK_CLIENT)

### LangSmith Tracing

In [53]:
import os
# from uuid import uuid4

# unique_id = uuid4().hex[0:8]
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__5512334fadb54637a91e0b644e843283"  # Update to your API key

### 함수 정의

In [54]:
import json
import requests
import pandas as pd

fitcloud_url = "https://aws-dev.fitcloud.co.kr"
corpId = "KDjAqAG0TnEAAAFK5eqDUL0A"

# account 별 월 별 사용량 (saving plan 포함)
# type: usage - ApplySavingsPlanCompute
def corp_month(
    start_month: str, 
    end_month: str,
    token: str,
    groupBy="account",
    ):
  api_url = fitcloud_url + "/service/trend/corp/month"
  cookies = {
    "JSESSIONID": token,
  }

  data = {
      "from": start_month,
      "to": end_month,
      "groupBy": groupBy,
  }

  resp = requests.post(api_url, json=data, cookies=cookies)

  if resp.status_code == 200:
    # 일반 월 사용량에서 SavingPlan 가격을 제함
    return pd.DataFrame(resp.json())

  else:
    print("error")
#------------------------------------------------------------------------------------
# 월 입력값이  from: '201901', to: '202210'형태일 경우
# 시작 월부터 종료 월까지 리스트로 출력
from datetime import datetime, timedelta

def month_range(start_month, end_month):
    # Create datetime objects for start and end dates
    start_date = datetime.strptime(start_month, "%Y%m")
    end_date = datetime.strptime(end_month, "%Y%m")

    # Initialize list to store months
    months_list = []

    # Iterate over months and add them to the list
    current_month = start_date
    while current_month <= end_date:
        months_list.append(current_month.strftime("%Y%m"))
        current_month = (current_month + timedelta(days=32)).replace(day=1)

    return months_list
#-------------------------------------------------------------------------------------
# account 일자별 사용량을 반환
def ondemand_account_day(
    accountId: str, 
    day_from: str, 
    day_to: str, 
    token: str) -> float:
  api_url = fitcloud_url + "/ondemand/account/day"
  cookies = {
    "JSESSIONID": token,
  }

  data = {
      "from": day_from,
      "to": day_to,
      "accountId": accountId,
  }
  resp = requests.post(api_url, json=data, cookies=cookies)

  if resp.status_code == 200:
    # JSON 형식으로 응답을 파싱 후 usageFee 합계를 구하기 위해 dataframe 의 변환
    df = pd.DataFrame(resp.json())
    usage_sum = round( df['usage_fee'].astype("Float32").sum(), 2)
    return usage_sum

  else:
    print("error")

# def corp_month_internal(start_month: str, end_month: str, accountId: str, token: str):
#   """calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars.
#   """
#   json_data = corp_month(start_month, end_month, token)
#   df = pd.DataFrame(json_data)
#   # accountId = accountId
#   # account에 관련된 데이터 추출
#   # df = df.query("accountId==@accountId")
#   df = df[df['accountId'] == accountId]
#   # 기간 내 월 리스트 추출
#   month_list = month_range(start_month, end_month)
#   # 월 column의 data type을 numeric으로 변환
#   df_acc = df.copy()
#   df_acc[month_list] = df_acc[month_list].apply(pd.to_numeric)
#   # 내부 사용자용 filter: 합산에 포함시킬 항목
#   internal_filter = ['Usage','ApplySavingsPlanCompute', 'ApplyRI' ]
#   # internal_filter = ['Usage','ApplySavingsPlanCompute']
#   df_int = df_acc.query("type in @internal_filter")
#   sum = df_int[month_list].sum().sum()
#   return sum

   

### 툴 정의

In [55]:
from crewai_tools import BaseTool

class UsageOfAmount(BaseTool):
    name: str = "get_usage"
    description: str = "calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars."

    def _run(self, start_month: str, end_month: str, accountId: str, token: str ):
        """calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars.
         """
        json_data = corp_month(start_month, end_month, token)
        df = pd.DataFrame(json_data)
        # accountId = accountId
        # account에 관련된 데이터 추출
        # df = df.query("accountId==@accountId")
        df = df[df['accountId'] == accountId]
        # 기간 내 월 리스트 추출
        month_list = month_range(start_month, end_month)
        # 월 column의 data type을 numeric으로 변환
        df_acc = df.copy()
        df_acc[month_list] = df_acc[month_list].apply(pd.to_numeric)
        # 내부 사용자용 filter: 합산에 포함시킬 항목
        internal_filter = ['Usage','ApplySavingsPlanCompute', 'ApplyRI' ]
        # internal_filter = ['Usage','ApplySavingsPlanCompute']
        df_int = df_acc.query("type in @internal_filter")
        sum = df_int[month_list].sum().sum()
        return round(sum, 2)
    
get_usage = UsageOfAmount()

#### Define Tasks

In [56]:
# Define your agents with roles and goals
from crewai import Agent
cost_expert = Agent(
    role="FitCloud Cost Expert",
    goal='calculate resource usage per account',
    backstory="""You are AWS resource cost expert""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[ get_usage ]
)

In [57]:
# Create tasks for your agents
from crewai import Task

accountId = "532805286864"
token = "5E20675268756DB4C7052065D7797949"

import datetime

year = datetime.date.today().year
month = datetime.date.today().month
day = datetime.date.today().day


task1 = Task(
    description=("""
        accountId는 {accountId} 이고, token은 {token}입니다.
        start_month와 end_month format은 '%Y%m'입니다.
        오늘은 {year}년 {month}월 {day}일 입니다.
        year 정보가 없을 경우 <year></year>사이의 정보를 사용하세요
        <year>{year}</year>
        month 정보가 하나일 경우, start_month와 end_month는 동일합니다.
        지난 달의 의미 previous month 이고, 작년의 의미는 year-1 입니다.
        답변 시 반드시 한국어를 사용하고 계정(account) 정보는 사용하지 마세요.
        start_month와 end_month는 string type으로 별도 변환이 필요 없습니다.
          {user_input}"""),
    expected_output='FitCloud 시스템에서 account가 사용한 aws resource cost를 정확히 계산하여 알려준다.',
    tools = [ get_usage],
    agent=cost_expert,
    output_file='task1.txt',
)

### Form Crew

In [58]:
from crewai import Crew, Process

# Forming the tech-focused crew with enhanced configurations
crew = Crew(
  agents=[cost_expert],
  tasks=[task1],
  process=Process.sequential  # Optional: Sequential task execution is default
)

2024-03-17 22:00:51,254 - 140096327742528 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


#### Kick it Off

In [59]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'accountId': accountId, 
                              'token': token ,
                              'year': year,
                              'month': month,
                              'day': day,
                              'user_input': '9월 사용량은?'})
print(result)



> Entering new CrewAgentExecutor chain...


Thought: 9월 사용량을 계산하기 위해서는 start_month와 end_month를 202409로 설정하고 get_usage 액션을 사용해야 합니다.

Action: get_usage
Action Input: {"start_month": "202409", "end_month": "202409", "accountId": "532805286864", "token": "5E20675268756DB4C7052065D7797949"}
 

I encountered an error while trying to use the tool. This was the error: 'accountId'.
 Tool get_usage accepts these inputs: get_usage(start_month: 'string', end_month: 'string', accountId: 'string', token: 'string') - calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars.

Thought: 9월 사용량을 계산하기 위해서는 start_month와 end_month를 202409로 설정하고 get_usage 액션을 사용해야 합니다.

Action: get_usage
Action Input: {"start_month": "202409", "end_month": "202409", "accountId": "532805286864", "token": "5E20675268756DB4C7052065D7797949"}
 

I encountered an error while trying to use the tool. This was the error: 'accountId'.
 Tool get_usage accepts these inputs: get_usage(start_month: 'string',